In [ ]:
import regex as re
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')

tsv = open(f"alt.atheism.tsv").readlines()

items = [row.replace("\n", '').split('\t')[0].lower() for row in tsv]

print('total:', len(items))
print(items[:5])



items = [x for x in items if not re.match('[^\P{P}-]+', x)]
print('\nafter regex:', len(items))
print(items[:5])


stop_words = set(stopwords.words('english'))
items = [w for w in items if not w.lower() in stop_words]
print('\nafter removing stop words:', len(items))
print(items[:5])

In [ ]:
def make_3_gram(items):
    _3_grams = []

    for _1, _2, _3 in zip(*[iter(items)]*3):
        _3_grams.append((_1, _2, _3))

    return _3_grams

_3_grams = make_3_gram(items[0:6]) + make_3_gram(items[1:6]) + make_3_gram(items[2:6])
_3_grams




In [ ]:
from operator import itemgetter

frequency_cache = {}
frequency_cache_n_grams = {}

def get_frequency(frequency_cache, item, items):
    if item not in frequency_cache:
        frequency_cache[item] = items.count(item)
        
    return frequency_cache[item]

def t_score(items):
    _3_grams = make_3_gram(items[0:]) + make_3_gram(items[1:]) + make_3_gram(items[2:])
    #print(len(_3_grams))
    result = []

    for _1, _2, _3 in _3_grams:
        frequency = get_frequency(frequency_cache_n_grams, (_1,_2,_3), _3_grams) #_3_grams.count((_1, _2, _3))
        _1_frequency = get_frequency(frequency_cache, _1, items)
        _2_frequency = get_frequency(frequency_cache, _2, items)
        _3_frequency = get_frequency(frequency_cache, _3, items)
        #print(len(frequency_cache))
        t_score = (frequency - ((_1_frequency * _2_frequency * _3_frequency) / len(items) ** 2)) / (frequency ** 0.5)
        result.append((t_score, _1, _2, _3))

    return result
        
t_scored = sorted(t_score(items[:]), key = itemgetter(0), reverse=True)
t_scored = list(dict.fromkeys(t_scored))

In [ ]:
t_scored[:30]

In [ ]:
import math 

def mi_score(items):
    _3_grams = make_3_gram(items[0:]) + make_3_gram(items[1:]) + make_3_gram(items[2:])
    
    result = []

    for _1, _2, _3 in _3_grams:
        frequency = get_frequency(frequency_cache_n_grams, (_1,_2,_3), _3_grams)
        _1_frequency = get_frequency(frequency_cache, _1, items)
        _2_frequency = get_frequency(frequency_cache, _2, items)
        _3_frequency = get_frequency(frequency_cache, _3, items)
        
        mi_score = math.log2((frequency * (len(items)**2))/(_1_frequency * _2_frequency * _3_frequency))
        result.append((mi_score, _1, _2, _3))

    return result
        
mi_scored = sorted(mi_score(items[:]), key = itemgetter(0), reverse=True)
mi_scored = list(dict.fromkeys(mi_scored))

In [ ]:
mi_scored[:30]

In [ ]:
import nltk
from nltk.collocations import *
from nltk.corpus import PlaintextCorpusReader

trigram_measures = nltk.collocations.TrigramAssocMeasures()

tokens = [row.replace("\n", '').split('\t')[0].lower() for row in tsv]

text = nltk.Text(tokens)

finder_thr = TrigramCollocationFinder.from_words(text)


finder_thr.nbest(trigram_measures.pmi, 30)